<a href="https://colab.research.google.com/github/BaseKan/aiday_training_resources/blob/main/Cython/main_solutions.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Setup

In [ ]:
%load_ext Cython
import numpy as np
import pandas as pd
from math import sin,tan,cos

# Opdracht: np.vectorize naar Cython

In [ ]:
def complicated_calculation(x,y):
  if x > 0.5*y and y < 0.3:
      res = sin(x-y)
  elif x < 0.5*y:
      res = tan(x+y)
  elif x > 0.2*y:
      res = sin(x)*np.sin(y)
  else:
      res = cos(x/(0.1+abs(y)))
  return res

In [ ]:
def get_results_fast(x,y):
  return np.vectorize(complicated_calculation)(x,y)

In [ ]:
x = np.random.randn(int(1e6))
y = np.random.randn(int(1e6))

In [ ]:
%timeit res_fast = get_results_fast(x, y)

In [ ]:
%%cython --annotate
cimport cython
import numpy as np
cimport numpy as np
from libc.math cimport sin, cos, tan, fabs

@cython.boundscheck(False)
@cython.wraparound(False)
cdef double complicated_calculation(double x, double y):
  cdef double res
  if x > 0.5*y and y < 0.3:
      res = sin(x-y)
  elif x < 0.5*y:
      res = tan(x+y)
  elif x > 0.2*y:
      res = sin(x)*sin(y)
  else:
      res = cos(x/(0.1+fabs(y)))
  return res

@cython.boundscheck(False)
@cython.wraparound(False)
def c_get_results_fast(double[:] x, double[:] y):
  cdef int n
  n = len(x)
  cdef double[:] res = np.zeros(n, dtype=np.dtype('d'))
  for i in range(n):
    res[i] = complicated_calculation(x[i],y[i])
  
  return res

In [ ]:
%timeit res_fast = c_get_results_fast(x, y)

# Opdracht:

In [ ]:
!curl -L -c cookies.txt 'https://docs.google.com/uc?export=download&id=151gCztjHR_D2uIoebxfi52DZWGLabOQd' | sed -rn 's/.*confirm=([0-9A-Za-z_]+).*/\1/p' > confirm.txt    
!curl -L -b cookies.txt -o 'weatherAUS.zip' 'https://docs.google.com/uc?export=download&id=151gCztjHR_D2uIoebxfi52DZWGLabOQd&confirm='$(<confirm.txt)
!unzip weatherAUS.zip
!rm -f confirm.txt cookies.txt weatherAUS.zip

In [ ]:
df = pd.read_csv('weatherAUS.csv')